In [6]:
class Graph:
    def __init__(self):
        self.adj_dict = {}
        
    def add_edge(self, v, u) :
        if v not in self.adj_dict:
            self.adj_dict[v] = {u:1}
        else:
            self.adj_dict[v][u] = 1
            
        if u not in self.adj_dict:
            self.adj_dict[u] = {v:1}
        else:
            self.adj_dict[u][v] = 1
        
    def adj(self, v):
        return self.adj_dict[v]
    
    def vertex(self, v):
        self.adj_list = self.adj_dict[v]
        self.deg = sum(list(map(int, self.adj_list.values())))
        self.vertex_num = 1
        
        return self.vertex_num, self.adj_list, self.deg

In [50]:
import networkx as nx
# import sys
from pyvis.network import Network
import matplotlib.pyplot as plt
import time

# edges_path = './dataset/ENZYMES118/ENZYMES118.edges' #sys.argv[1]
# # edges_path = './dataset/soc-YouTube-ASU/soc-YouTube-ASU.edges' #sys.argv[1]
# label_path = './dataset/ENZYMES118/ENZYMES118.node_labels' #sys.argv[2]
edges_path = './dataset/NCI1/NCI1.edges' #sys.argv[1]
label_path = './dataset/NCI1/NCI1.node_labels' #sys.argv[2]
# k = 7 #sys.argv[3]
label_dict = {}
graph_sort = []
label_kind = []
node_dict = {}
superNode = {}
label_f = open(label_path, 'r')
while True:
    line = label_f.readline().replace('\n', '').strip()
    if not line:
        label_kind.append('Noen')
        break
    node = line.split(',')[0]
    label = line.split(',')[1]
    label_dict[node] = label
    label_kind.append(label)
label_f.close

label_kind = list(set(label_kind))
print(label_kind)

edge_f = open(edges_path, 'r')
gs = Graph()
while True:
    line = edge_f.readline().replace('\n', '').strip()
    if not line:
        break
    
    v = line.split(',')[0]
    u = line.split(',')[1]
    
    if v not in node_dict:
        node_dict[v] = v
    if u not in node_dict:
        node_dict[u] = u
    
    if v not in label_dict:
        label_dict[v] = 'None'
        
    if u not in label_dict:
        label_dict[u] = 'None'
    graph_sort.append([v,u])
    gs.add_edge(v,u)
edge_f.close

['36', '30', '11', '18', '16', '15', '1', '19', '23', '35', 'Noen', '22', '27', '34', '37', '29', '20', '14', '4', '17', '13', '24', '33', '7', '25', '10', '6', '3', '28', '5', '32', '12', '21', '31', '9', '2', '8', '26']


<function TextIOWrapper.close()>

In [44]:
G = nx.Graph()
for v in node_dict:
    G.add_node(v, title = v)
    
for v, u in graph_sort:
    G.add_edge(v,u)

In [51]:
def Super_node_generator(v):
    num_v, adj_v, deg_v = gs.vertex(v)
    label_v = {}
    
    for l in label_kind:
        label_v[l] = 0
        
    if v in label_dict:
        for ld_v in label_v:
            if ld_v == label_dict[v]:
                label_v[ld_v] = 1
    
    Vi = [num_v, deg_v, adj_v, label_v]
    
    return Vi

def Super_node_similarity(v, Vi, u, Vj):

    ew_v = Vi[2][u]/Vi[1]
    ew_u = Vj[2][v]/Vj[1]
    n_max = max(Vi[0], Vj[0])
    d_max = max(Vi[1], Vj[1])

    lm_v = sum(list(map(int, Vi[3].values())))
    lm_u = sum(list(map(int, Vj[3].values())))
    dis_att = 0
    
    dis_st = (abs(Vi[0]-Vj[0])/n_max) + (abs(Vi[1]-Vj[1])/d_max) + abs((Vi[2][u]/Vi[1]) - (Vj[2][v]/Vj[1]))/3

    for vl in Vi[3]:
        dis_att += abs(Vi[3][vl]/lm_v - Vj[3][vl]/lm_u)/2

    dis = (dis_st + dis_att)/2
    sim = 1 - dis
    return sim

def Super_node_similarity2(v, Vi, u, Vj, n):
    
    ew_v = Vi[2][u]/Vi[1]
    
    if v not in Vj[2] or n in Vj[2]:
        save_val = Vj[2][n]
        del Vj[2][n]
        Vj[2][v] = save_val
    
    ew_u = Vj[2][v]/Vj[1]
    n_max = max(Vi[0], Vj[0])
    d_max = max(Vi[1], Vj[1])

    lm_v = sum(list(map(int, Vi[3].values())))
    lm_u = sum(list(map(int, Vj[3].values())))
    dis_att = 0
    
    dis_st = (abs(Vi[0]-Vj[0])/n_max) + (abs(Vi[1]-Vj[1])/d_max) + abs((Vi[2][u]/Vi[1]) - (Vj[2][v]/Vj[1]))/3

    for vl in Vi[3]:
        dis_att += abs(Vi[3][vl]/lm_v - Vj[3][vl]/lm_u)/2

    dis = (dis_st + dis_att)/2
    sim = 1 - dis
    return sim

def Merged_super_node(v, u, Vi, Vj):
    num_v = Vi[0]+Vj[0]
    adj_v = {}
    label_v = {}
    for adj in Vi[2]:
        if adj == u:
            continue
        if adj in Vj[2]:
            adj_v[adj] = Vi[2][adj] + Vj[2][adj]
        else:
            adj_v[adj] = Vi[2][adj]
            
    for adj in Vj[2]:
        if adj == v:
            continue
        if adj not in Vi[2]:
            adj_v[adj] = Vj[2][adj]
    if v in adj_v:
        del adj_v[v]
        
    if u in adj_v:
        del adj_v[u]
    deg_v = sum(list(map(int, adj_v.values())))
    
    for lab in Vi[3]:
        if lab in Vj[3]:
            label_v[lab] = Vi[3][lab] + Vj[3][lab]
            
    nVi = [num_v, deg_v, adj_v, label_v]
    
    return nVi

In [ ]:
for node in node_dict:
    print(node)
    superNode[node] = Super_node_generator(node)
    
sim_list = []

for v, u in graph_sort:
    sim_list.append([v, u, Super_node_similarity(v, superNode[v], u, superNode[u])]) #triple entry
   
sim_list.sort(reverse=True, key=lambda x:x[2])

In [ ]:
print(label_dict['37140'])
print(label_kind)

In [54]:
print(len(sim_list))
c = 0
while(True): #k
    v = sim_list[0][0]
    u = sim_list[0][1]
    sim = sim_list[0][2]
    s1 = time.time()
    G = nx.Graph()
    
    if sim <= 0: #48
        break
        
    if v in superNode and u in superNode:
        # print(f'v:{v}, u:{u}')
        # print('superNode: v => ')
        # print(superNode[v])
        # print('superNode: u => ')
        # print(superNode[u])
        del_list = []
        superNode[v] = Merged_super_node(v, u, superNode[v], superNode[u])
        # print('merged superNode:v =>')
        # print(superNode[v])
        # print('del superNode:u =>')
        del(superNode[u])
        # print('sim_list0: ')
        # print(sim_list[0])
        del(sim_list[0])
        
        nei_e = list(superNode[v][2].keys())
        # print('v의 이웃노드: ')
        # print(nei_e)
        for sim_one in sim_list:
            if sim_one[0] == v or sim_one[0] == u or sim_one[1] == v or sim_one[1] == u:
                # print('v,u의 포함 엣지 탐색: ')
                # print(sim_one)
                del_list.append(sim_one)
                
        for del_one in del_list:
            sim_list.remove(del_one)

        for n in nei_e:
            # print('이웃노드 n: ')
            # print(n)
            if n not in superNode:
                # print('이미 삭제된 노드')
                del superNode[v][2][n]
                continue
            else:
                # print('유사도 측정하고 추가')
                # print(f'v:{v}, n:{n},유사도')
                # print(Super_node_similarity2(v, superNode[v], n, superNode[n], u))
                
                sim_list.append([v, n, Super_node_similarity2(v, superNode[v], n, superNode[n], u)])
                
        # print('Size')
        # print(len(sim_list))

    else:
        # print('없는 슈퍼 노드')
        del(sim_list[0])
        
    for v1 in superNode:
        G.add_node(v1, label = v1)
        
    for i in range(len(sim_list)):
        v2 = sim_list[i][0]
        u2 = sim_list[i][1]
        G.add_edge(v2,u2)
        
    graph_name = 'NCI1_'+ str(c) +'.html'
    net = Network(bgcolor="#222222", font_color="white", filter_menu=True, select_menu=True)
    net.from_nx(G)

    net.show_buttons(filter_=['physics'])
    net.show(graph_name)

    sim_list.sort(reverse=True, key=lambda x:x[2])
    deg_size = len(nei_e)
    sim_size = len(sim_list)
    e1 = time.time()
    d1 = e1-s1
    print(f'c: {c} v: {v}, u: {u}, Size: {sim_size}, Time: {d1}, deg: {deg_size}')
    c = c + 1
    
    if sim_size <= 3: #48
        break


265502


KeyboardInterrupt: 

In [250]:
Gs = sim_list
f = open('result_ENZYMES118.edges', 'w')

for e in Gs:
    str_e = e[0] + ',' + e[1] + '\n'
    f.write(str_e)
        
f.close()

In [251]:
print(superNode.keys())
print(sim_list)

dict_keys(['10', '63', '66', '14', '58', '70', '73', '23', '28', '80', '82', '86', '37', '84', '87', '89', '93', '46', '53'])
[['86', '37', -0.08333333333333348], ['80', '28', -0.08333333333333348], ['63', '89', -0.09508547008547019], ['93', '46', -0.11904761904761907], ['63', '80', -0.12606837606837606], ['63', '53', -0.16239316239316226], ['63', '14', -0.16239316239316226], ['63', '23', -0.16239316239316226], ['63', '93', -0.18032661782661785], ['63', '10', -0.2735042735042734], ['63', '82', -0.27831196581196593], ['63', '58', -0.3257478632478632], ['63', '84', -0.36431623931623935], ['63', '70', -0.36431623931623935], ['63', '86', -0.3712606837606838], ['63', '66', -0.37393162393162394], ['63', '87', -0.37393162393162394], ['63', '73', -0.37393162393162394]]


In [252]:
for sn in superNode:
    print(sn, superNode[sn])

10 [1, 1, {'63': 1}, {'0': 0, '2': 0, '1': 1}]
63 [52, 24, {'66': 2, '10': 1, '87': 2, '84': 2, '86': 1, '70': 2, '73': 2, '58': 1, '53': 2, '89': 2, '14': 2, '82': 1, '80': 1, '23': 2, '93': 1}, {'0': 0, '2': 15, '1': 37}]
66 [1, 2, {'63': 1}, {'0': 0, '2': 1, '1': 0}]
14 [1, 2, {'63': 1}, {'0': 0, '2': 0, '1': 1}]
58 [5, 2, {'63': 2}, {'0': 0, '2': 4, '1': 1}]
70 [2, 2, {'63': 1}, {'0': 0, '2': 2, '1': 0}]
73 [1, 2, {'63': 1}, {'0': 0, '2': 1, '1': 0}]
23 [1, 2, {'63': 1}, {'0': 0, '2': 0, '1': 1}]
28 [1, 1, {'80': 1}, {'0': 0, '2': 0, '1': 1}]
80 [8, 2, {'28': 1, '63': 1}, {'0': 0, '2': 5, '1': 3}]
82 [3, 2, {'63': 2}, {'0': 0, '2': 2, '1': 1}]
86 [2, 2, {'37': 1, '63': 1}, {'0': 0, '2': 2, '1': 0}]
37 [1, 1, {'86': 1}, {'0': 0, '2': 0, '1': 1}]
84 [2, 2, {'63': 1}, {'0': 0, '2': 2, '1': 0}]
87 [1, 2, {'63': 1}, {'0': 0, '2': 1, '1': 0}]
89 [4, 2, {'63': 1}, {'0': 0, '2': 2, '1': 2}]
93 [7, 2, {'46': 1, '63': 1}, {'0': 0, '2': 5, '1': 2}]
46 [1, 1, {'93': 1}, {'0': 0, '2': 0, '1': 1

In [ ]:
    
Vi = [num_v, deg_v, adj_v, label_v] #0: 노드 수, 엣지 수, 인접리스트, 라벨

ew_v = Vi[2][u]/Vi[1] # u의 엣지수/전체 엣지 수
ew_u = Vj[2][v]/Vj[1]
n_max = max(Vi[0], Vj[0]) #큰 노드 수
d_max = max(Vi[1], Vj[1]) #큰 차수 수

lm_v = sum(list(map(int, Vi[3].values())))
lm_u = sum(list(map(int, Vj[3].values())))
dis_att = 0

dis_st = ((abs(Vi[0]-Vj[0])/n_max) + (abs(Vi[1]-Vj[1])/d_max) + abs((Vi[2][u]/Vi[1]) - (Vj[2][v]/Vj[1])))/3

for vl in Vi[3]:
    dis_att += abs(Vi[3][vl]/lm_v - Vj[3][vl]/lm_u)/2

dis = (dis_st + dis_att)/2
sim = 1 - dis